In [1]:
import PyPDF2
import requests
import io
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
pths = '../data/links.csv'

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(pths)
df

,TITULO,Link
0,7DIESACTUALITAT.COM,https://www.ojdinteractiva.es/muestra_acta/med...
1,7TELEVALENCIA.COM,https://www.ojdinteractiva.es/muestra_acta/med...
2,ACTUALIDADCASTELLON.COM,https://www.ojdinteractiva.es/muestra_acta/med...
3,ACTUALIDADVALENCIA.COM,https://www.ojdinteractiva.es/muestra_acta/med...
4,AGENDADEISA.COM,https://www.ojdinteractiva.es/muestra_acta/med...
...,...,...
333,URGELLTV.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...
334,21RADIO.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...
335,AUE.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...
336,FEMBASE.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...


In [5]:
def get_pdf_text_from_link(link):
    '''get pdf text from link of pdf'''
    response = requests.get(link)
    pdf = PyPDF2.PdfReader(io.BytesIO(response.content))
    text = ''
    for i in range(len(pdf.pages)):
        text += pdf.pages[i].extract_text()
    return text


In [6]:
# pdf_texts = df['Link'].apply(get_pdf_text_from_link)
for i in tqdm(range(len(df))):
    try:
        df.loc[i, 'Text'] = get_pdf_text_from_link(df.loc[i, 'Link'])
    except:
        df.loc[i, 'Text'] = np.nan
        print('Error at', i)

  0%|          | 0/338 [00:00<?, ?it/s]

100%|██████████| 338/338 [17:19<00:00,  3.08s/it]


In [7]:
# extract zipcode from this pattern : Ciudad: 46002 VALENCIA, starts with Ciudad: , then 5 or 6 digits
def extract_zipcode(text):
    '''extract zipcode from text'''
    pattern = r'Ciudad: \d{5,6}'
    zipcode = re.search(pattern, text)
    if zipcode:
        return zipcode.group().split(' ')[1]
    else:
        pattern = r'Ciutat: \d{5,6}'
        zipcode = re.search(pattern, text)
        if zipcode:
            return zipcode.group().split(' ')[1]
        else:
            return np.nan


In [8]:
df

,TITULO,Link,Text
0,7DIESACTUALITAT.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...
1,7TELEVALENCIA.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...
2,ACTUALIDADCASTELLON.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...
3,ACTUALIDADVALENCIA.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...
4,AGENDADEISA.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...
...,...,...,...
333,URGELLTV.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...
334,21RADIO.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...
335,AUE.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...
336,FEMBASE.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...


In [9]:
zips = df['Text'].apply(extract_zipcode)

In [10]:
zips

0      12500
1      46988
2      46007
3      46007
4      46112
       ...  
333    25007
334    03500
335    25007
336    25001
337    08007
Name: Text, Length: 338, dtype: object

In [11]:
df['zipcode'] = zips

In [12]:
df[df.zipcode.isna()]

,TITULO,Link,Text,zipcode
7,ALICANTEPLAZA.ES,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...,NaN
211,DIARIANDORRA.AD,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...,NaN
262,ELPERIÒDIC.AD,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...,NaN


In [13]:
df.rename(columns={'Link': 'pdf_link', 'TITULO': 'media' }, inplace=True)

In [14]:
df[['media','pdf_link', 'Text', 'zipcode']].to_csv('../data/links_with_zipcodes_Text.csv', index=False)